<a href="https://colab.research.google.com/github/componavt/LLLE-R1900s/blob/main/src/visualization/grouped_bar_chart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📊 Grouped Bar Chart: Annual Loan Volumes & Counts by Credit Type

Visualizes **annual dynamics** of selected credit categories across all settlements.

**Top bars (↑)** = total loan amount (in **thousands of rubles**)  
**Bottom bars (↓)** = total number of loans  

Each selected credit type is shown with:
- **Unique color**
- **Consistent style** for both amount and count (same color for ↑ and ↓)

✨ **Features:**
- 🌍 Fully bilingual: toggle between Russian (`ru`) and English (`en`)
- 📅 X-axis = years (e.g., 1913–1917)
- 🎨 Configurable credit types via `SELECTED_CREDIT_ITEMS`
- 📈 Dual-axis: positive (amount) and negative (count) bars share color per category
- 🖼️ Rendered directly in Google Colab

⚙️ Configure via:
- `USE_LANGUAGE` — `"ru"` or `"en"`
- `SELECTED_CREDIT_ITEMS` — list of `Name` values from `credit_items.csv`

⚙️ [2] Install Dependencies & Set Language + Credit Selection

In [1]:
# Install compatible versions for static image rendering in Colab
!pip install -q python-dotenv pandas plotly

# Clone the repo if running in Colab (optional — if data not uploaded manually)
import os

if not os.path.exists('LLLE-R1900s'):
    !git clone https://github.com/componavt/LLLE-R1900s.git
    %cd LLLE-R1900s
else:
    %cd LLLE-R1900s

# === LANGUAGE SWITCH ===
USE_LANGUAGE = "ru"
assert USE_LANGUAGE in ("ru", "en"), "USE_LANGUAGE must be 'ru' or 'en'"

# === SELECT CREDIT TYPES TO VISUALIZE ===
# Must use values from the 'Name' column in credit_items.csv
SELECTED_CREDIT_ITEMS = ["Migration", "CraftMaterials", "CraftTools"]

# Validate selection
valid_names = set(pd.read_csv('data/credit_items.csv')['Name'])
invalid = set(SELECTED_CREDIT_ITEMS) - valid_names
if invalid:
    raise ValueError(f"Invalid credit item(s): {invalid}. Must be from 'Name' column in credit_items.csv.")

Cloning into 'LLLE-R1900s'...
remote: Enumerating objects: 547, done.
remote: Counting objects: 100% (547/547), done.
remote: Compressing objects: 100% (245/245), done.
remote: Total 547 (delta 409), reused 401 (delta 296), pack-reused 0 (from 0)
Receiving objects: 100% (547/547), 1.40 MiB | 13.11 MiB/s, done.
Resolving deltas: 100% (409/409), done.
/content/LLLE-R1900s


📥 [3] Load & Prepare Data

In [2]:
📥 [3] Load & Prepare Minimal Data for Bar Chart

import os
import pandas as pd
from dotenv import load_dotenv

# Load configuration
load_dotenv('config.env')

# Paths
csv_out_dir = os.getenv('CSV_OUT_DIR', 'data/csv_out')
output_file_name = os.getenv('OUTPUT_CSV_FILE')

if not output_file_name:
    csv_files = [f for f in os.listdir(csv_out_dir) if f.endswith('.csv')]
    if not csv_files:
        raise FileNotFoundError("No CSV files found in the output directory.")
    output_file_name = csv_files[0]

csv_path = os.path.join(csv_out_dir, output_file_name)
print(f"Loading loan data from: {csv_path}")

# Load main loan data
df_loans = pd.read_csv(csv_path)
print(f"Loaded {len(df_loans)} loan records.")

# Load credit items
df_credit = pd.read_csv('data/credit_items.csv')
print(f"Loaded {len(df_credit)} credit item definitions.")

# --- Build credit item display label map ---
if USE_LANGUAGE == "ru":
    df_credit['display_label'] = df_credit['loan_short_ru']
else:
    df_credit['display_label'] = df_credit['Name']

label_map = dict(zip(df_credit['Name'], df_credit['display_label']))

# Validate selected credit items
unknown_items = set(SELECTED_CREDIT_ITEMS) - set(df_credit['Name'])
if unknown_items:
    print("⚠️ Warning: Unknown credit_item in SELECTED_CREDIT_ITEMS:", unknown_items)

# Filter loans to only selected credit items
df_loans = df_loans[df_loans['credit_item'].isin(SELECTED_CREDIT_ITEMS)].copy()

# Ensure numeric types
df_loans['amount_rubles'] = pd.to_numeric(df_loans['amount_rubles'], errors='coerce').fillna(0)
df_loans['loan_count'] = pd.to_numeric(df_loans['loan_count'], errors='coerce').fillna(0).astype(int)

print(f"Filtered to {len(df_loans)} records for selected credit types: {SELECTED_CREDIT_ITEMS}")

Loading loan data from: data/csv_out/loans_s28_i21.csv
Loaded 1768 loan records.
Loaded 21 credit item definitions.
Loaded 29 settlements.
✅ Data prepared with full 3×3 credit profile matrix.


📊 [5] Grouped Bar Chart: Annual Amount (↑) and Count (↓) by Credit Type

In [ ]:
📊 [5] Grouped Bar Chart: Annual Loan Amount and Count

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Aggregate by year and credit_item
df_annual = df_loans.groupby(['year', 'credit_item']).agg(
    total_amount=('amount_rubles', 'sum'),
    total_count=('loan_count', 'sum')
).reset_index()

# Convert amount to thousands for readability
df_annual['total_amount_k'] = df_annual['total_amount'] / 1000.0

# Get unique years and credit items (in selected order)
years = sorted(df_annual['year'].unique())
credit_items = SELECTED_CREDIT_ITEMS  # preserve user-defined order

# Prepare data for plotting
amount_data = {}
count_data = {}
for item in credit_items:
    subset = df_annual[df_annual['credit_item'] == item]
    amount_dict = dict(zip(subset['year'], subset['total_amount_k']))
    count_dict = dict(zip(subset['year'], subset['total_count']))
    amount_data[item] = [amount_dict.get(y, 0) for y in years]
    count_data[item] = [-count_dict.get(y, 0) for y in years]  # negative for downward bars

# Create figure
fig = go.Figure()

# Define colors (Plotly default palette)
colors = plotly.colors.qualitative.Plotly

# Add traces for amount (positive) and count (negative)
for i, item in enumerate(credit_items):
    color = colors[i % len(colors)]
    label = label_map[item]

    # Amount (upward bars)
    fig.add_trace(go.Bar(
        x=years,
        y=amount_data[item],
        name=f"{label} — Amount" if USE_LANGUAGE == "en" else f"{label} — Сумма",
        marker_color=color,
        offsetgroup=i,
        legendgroup=item
    ))

    # Count (downward bars)
    fig.add_trace(go.Bar(
        x=years,
        y=count_data[item],
        name=f"{label} — Count" if USE_LANGUAGE == "en" else f"{label} — Кол-во",
        marker_color=color,
        opacity=0.7,
        offsetgroup=i,
        legendgroup=item,
        showlegend=False  # avoid duplicate legend entries
    ))

# Titles and labels
if USE_LANGUAGE == "en":
    title = "Annual Loan Volume by Credit Type"
    yaxis_title = "Amount (thsd rub) ↑ / Number of Loans ↓"
    hover_amount = "Amount: %{y:.1f}k rub"
    hover_count = "Loans: %{y:.0f}"
else:
    title = "Годовой объём ссуд по категориям"
    yaxis_title = "Сумма (тыс. руб.) ↑ / Число ссуд ↓"
    hover_amount = "Сумма: %{y:.1f} тыс. руб."
    hover_count = "Ссуд: %{y:.0f}"

fig.update_layout(
    title=title,
    barmode='group',
    xaxis=dict(title="Year" if USE_LANGUAGE == "en" else "Год", tickmode='linear'),
    yaxis=dict(title=yaxis_title),
    legend_title="Credit Type" if USE_LANGUAGE == "en" else "Категория кредита",
    height=600,
    font=dict(size=12)
)

# Show in Colab
fig.show(renderer="colab")

print(f"\n✅ Displayed grouped bar chart for {len(credit_items)} credit types in {'English' if USE_LANGUAGE == 'en' else 'Russian'}.")

🌟 [4] Individual Settlement Charts

📓 Ячейка 4: Отдельные диаграмма по всем поселениям (ещё карту бы...)